$(\textbf{x}, t)\in\Omega\times[0,\infty)$

$$\frac{\partial u}{\partial t} = \nabla^2u + 1 - u$$

$\textbf{x}\in\partial\Omega$ Dirichlet boundary condition

$$u = u_{\text{D}}(t)$$

$\textbf{x}\in\partial\Omega_{\text{N}}=\partial\Omega/\partial\Omega_{\text{D}}$ Neumann boundary condition

$$\textbf{n}\cdot\nabla{u} = u_{\text{N}}(t)$$

### $d=1$

$$\Omega = [0, L_x]$$

$$u_{\text{D}}(x=0, t)=\epsilon t$$

$$u_{\text{D}}(x=L_x)=0$$

In [ ]:
import numpy as np
from ufl import Form, dx, inner, grad, TestFunction

from lucifex.mesh import interval_mesh
from lucifex.fdm import (
    DT, BE,
    FiniteDifference, FunctionSeries, ConstantSeries,
)
from lucifex.sim import integrate, Simulation
from lucifex.fem import LUCiFExConstant as Constant
from lucifex.solver import ibvp_solver, eval_solver, BoundaryConditions
from lucifex.viz import plot_line


def diffusion_reaction(
    u: FunctionSeries,
    dt: Constant | float,
    Dt_diff: FiniteDifference,
    Dt_reac: FiniteDifference,
    bcs: BoundaryConditions | None = None,
) -> list[Form]:
    v = TestFunction(u.function_space)
    Ft = v * DT(u, dt) * dx
    Fd = inner(grad(v), grad(Dt_diff(u))) * dx
    Fr = -v * (1 - Dt_reac(u)) * dx
    forms = [Ft, Fd, Fr]
    if bcs is not None:
        ds, neumann_data = bcs.boundary_data(u.function_space, 'neumann')
        F_neumann = sum([-v * uN * ds(i) for i, uN in neumann_data])
        forms.append(F_neumann)
    return forms


def time_dependent_bcs_simulation(
    Lx: float,
    Nx: int,
    dt: float,
    eps: float,
    Dt_diff: FiniteDifference,
    Dt_reac: FiniteDifference,
):
    order = max((Dt_diff.order, Dt_reac.order, 1))
    store = 1
    mesh = interval_mesh(Lx, Nx)

    t = ConstantSeries(mesh, name='t', ics=0.0)
    dt = Constant(mesh, dt, name='dt')
    u = FunctionSeries((mesh, 'P', 1), 'u', order, store, ics=0.0)
    uD = FunctionSeries((mesh, 'P', 1), 'uD', order, store, ics=0.0)

    bcs = BoundaryConditions(
        ('dirichlet', lambda x: x[0], uD[0]),
        ('dirichlet', lambda x: x[0] - Lx, 0.0),
    )

    uD_expr = lambda t: eps * t

    uD_solver = eval_solver(uD, uD_expr)(t[0])
    u_solver = ibvp_solver(diffusion_reaction, bcs=bcs)(u, dt, Dt_diff, Dt_reac)

    return Simulation([u_solver, uD_solver], t, dt)


